In [1]:
import os, sys, random, time
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

from util import read_pickle

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
case_num=1
seed = 0

In [ ]:
def recompute_at_dt_pdotw(pdpt_ins, truck_route, created_truck_yCycle, A_sol, Ab_sol, D_sol, Db_sol):
    edge_shortest = pdpt_ins['edge_shortest']
    constant = pdpt_ins['constant']
    cargo_list = pdpt_ins['cargo']
    truck_list = pdpt_ins['truck'] # truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']


    for truck_key, route_ in truck_route.items():
        for node_idx in range(1, len(route)-1):
            node_curr, node_pre = route[node_idx], route[node_idx-1]
            # AT[curr_node] = DT[prev_node] + edge[pre_node, curr_node]
            A_sol[(node_curr, truck_)] = D[(node_pre, truck_)] + edge_shortest[(node_pre, node_curr)]

            if truck_key in created_truck_yCycle.keys():  # if truck returns back to depot
                if node_curr == created_truck_yCycle[truck_][1]: # if node_curr is its destination
                    Db[(node_curr, truck_key)] = Ab[(node_curr, truck_key)] + constant['node_fixed_time']
                    for cargo_key, cargo_route_pair in cargo_route.values():
                        if truck_key in [route_pair[0] for route_pair in cargo_route_pair]:
                            Db[(node_curr, truck_key)] += int(np.ceil(constant['loading_variation_coefficient']*cargo_list[cargo_key][0]))
                    assert Db[(node_curr, truck_key)] <= truck_list[truck_key][-2]
                else:
                    D[(node_curr, truck_key)] = A[(node_curr, truck_key)] + constant['node_fixed_time']
                    for cargo_key, cargo_route_pair in cargo_route.values():
                        if truck_key in [route_pair[0] for route_pair in cargo_route_pair]:
                            D[(node_curr, truck_key)] += int(np.ceil(constant['loading_variation_coefficient']*cargo_list[cargo_key][0]))
                    assert D[(node_curr, truck_key)] <= truck_list[truck_key][-2]
            # if truck_ is a non-cycle truck
            else:
                D[(node_curr, truck_key)] = A[(node_curr, truck_key)] + constant['node_fixed_time']
                for cargo_key, cargo_route_pair in cargo_route.values():
                    if truck_key in [route_pair[0] for route_pair in cargo_route_pair]:
                        D[(node_curr, truck_key)] += int(np.ceil(constant['loading_variation_coefficient']*cargo_list[cargo_key][0]))
                assert D[(node_curr, truck_key)] <= truck_list[truck_key][-2]

        
        assert D[(route_[-1], truck_key)] <= truck_list[truck_key][-2]
        assert Db[(route_[-1], truck_key)] <= truck_list[truck_key][-2]

    return A_sol, Ab_sol, D_sol, Db_sol
